In [54]:
import sys
import os

sys.path.append(os.getcwd()[:-9])
from src.ops.operator import Operator
from src.ops.elemental import ElementalOperator
from src.ops.quarks import Quark
from src.ops.indexed import IndexedObject
from src.wick.contract import contract

In [74]:
q0=Quark(True,'u','s_0','c_0','t_3','x_3')
g0=IndexedObject('\\gamma_5',['s_0','s_1'])
q1=Quark(False,'d','s_1','c_0','t_3','x_3')

annihilate=Operator([ElementalOperator(+1,[g0],[q0,q1])])

elementals=[]
for pi2 in [['u',+1],['d',+1]]:
    for pi1 in [['u',+1],['d',+1]]:
        q2=Quark(True,pi2[0],'s_2','c_1','t_2','x_2')
        g1=IndexedObject('\\gamma_{\mu}',['s_2','s_3'])
        q3=Quark(False,pi2[0],'s_3','c_1','t_2','x_2')

        q4=Quark(True,pi1[0],'s_4','c_2','t_1','x_1')
        g2=IndexedObject('\\gamma_{\\nu}',['s_4','s_5'])
        q5=Quark(False,pi1[0],'s_5','c_2','t_1','x_1')

        q6=Quark(False,'u','s_6','c_3','t_0','x_0')
        g3=IndexedObject('\\gamma_5',['s_6','s_7'])
        q7=Quark(True,'d','s_7','c_3','t_0','x_0')
        elementals.append(ElementalOperator(pi2[1]*pi1[1],[g1,g2,g3],[q2,q3,q4,q5,q6,q7]))
        
create=Operator(elementals)

In [75]:
res=contract(annihilate,create)

In [76]:
len(res)

20

In [78]:
import copy

def latexStr(gamma):
    tmp=gamma.name.split('_')
    if(tmp[0]=='\\gamma'):
        return tmp[0] + '_{' + tmp[1] + '_{' + gamma.indices[0] + gamma.indices[1] + '}}'
    else:
        return str(tmp)
  
def platexStr(p):
    return p.name + '(' + p.ti.replace('t','x') + p.ti + '\mid ' + p.tf.replace('t','x') + p.tf + ')' + '_{\\substack{' + p.left_indices.s + '\\\\' + p.left_indices.c + '}' +  '\\substack{' + p.right_indices.s + '\\\\' + p.right_indices.c + '}}'
    
total=copy.deepcopy(res)

print(len(total))

for d in total:
    for p in d.props:
        p.name = 'D^{-1}'
        
for i in range(len(total)-1,-1,-1):
    for j in range(len(total)-1,-1,-1):
        if(i!=j):
            if((total[i].commuting==total[j].commuting) and (total[i].props==total[j].props)):
            #if(total[i]==total[j]):
                total[i].coef+=total[j].coef
                total.pop(j)
                break
for i in range(len(total)-1,-1,-1):
    if(total[i].coef==0):
        total.pop(i)
        
print(len(total))

20
12


In [86]:
for i,d in enumerate(total):
    #if(i==1 or i==6):
        tmp='d_{'+str(i)+'}='+str(d.coef) + ' '
        for p in d.props:
            tmp+=platexStr(p)
        for c in d.commuting:
            tmp+=latexStr(c)
        print(tmp+'\\\\')

d_{0}=1 D^{-1}(x_1t_1\mid x_3t_3)_{\substack{s_5\\c_2}\substack{s_0\\c_0}}D^{-1}(x_0t_0\mid x_2t_2)_{\substack{s_6\\c_3}\substack{s_2\\c_1}}D^{-1}(x_2t_2\mid x_1t_1)_{\substack{s_3\\c_1}\substack{s_4\\c_2}}D^{-1}(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{1}=1 D^{-1}(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_1}\substack{s_0\\c_0}}D^{-1}(x_1t_1\mid x_2t_2)_{\substack{s_5\\c_2}\substack{s_2\\c_1}}D^{-1}(x_0t_0\mid x_1t_1)_{\substack{s_6\\c_3}\substack{s_4\\c_2}}D^{-1}(x_3t_3\mid x_0t_0)_{\substack{s_1\\c_0}\substack{s_7\\c_3}}\gamma_{5_{s_0s_1}}\gamma_{5_{s_6s_7}}\gamma_{{\mu}_{s_2s_3}}\gamma_{{\nu}_{s_4s_5}}\\
d_{2}=1 D^{-1}(x_2t_2\mid x_3t_3)_{\substack{s_3\\c_1}\substack{s_0\\c_0}}D^{-1}(x_0t_0\mid x_2t_2)_{\substack{s_6\\c_3}\substack{s_2\\c_1}}D^{-1}(x_3t_3\mid x_1t_1)_{\substack{s_1\\c_0}\substack{s_4\\c_2}}D^{-1}(x_1t_1\mid x_0t_0)_{\substack{s_5\\c_2}\substack{s_7\\c_3}}\gamma_{5_

In [80]:
diagrams = {}

for i,d in enumerate(total):
    top = []
    d_str='d_' + str(i) + ':  '
    for p in d.props:
        d_str+=p.tf + '->' + p.ti + '   '
        top.append([p.tf.split('_')[1],p.ti.split('_')[1]])
    diagrams[i]=top
    print(d_str)

d_0:  t_3->t_1   t_2->t_0   t_1->t_2   t_0->t_3   
d_1:  t_3->t_2   t_2->t_1   t_1->t_0   t_0->t_3   
d_2:  t_3->t_2   t_2->t_0   t_1->t_3   t_0->t_1   
d_3:  t_3->t_2   t_2->t_0   t_1->t_1   t_0->t_3   
d_4:  t_3->t_1   t_2->t_3   t_1->t_0   t_0->t_2   
d_5:  t_3->t_1   t_2->t_2   t_1->t_0   t_0->t_3   
d_6:  t_3->t_0   t_2->t_2   t_1->t_3   t_0->t_1   
d_7:  t_3->t_0   t_2->t_3   t_1->t_2   t_0->t_1   
d_8:  t_3->t_0   t_2->t_3   t_1->t_1   t_0->t_2   
d_9:  t_3->t_0   t_2->t_1   t_1->t_3   t_0->t_2   
d_10:  t_3->t_0   t_2->t_1   t_1->t_2   t_0->t_3   
d_11:  t_3->t_0   t_2->t_2   t_1->t_1   t_0->t_3   


In [81]:
# d_0 -> z


In [82]:
diagrams

{0: [['3', '1'], ['2', '0'], ['1', '2'], ['0', '3']],
 1: [['3', '2'], ['2', '1'], ['1', '0'], ['0', '3']],
 2: [['3', '2'], ['2', '0'], ['1', '3'], ['0', '1']],
 3: [['3', '2'], ['2', '0'], ['1', '1'], ['0', '3']],
 4: [['3', '1'], ['2', '3'], ['1', '0'], ['0', '2']],
 5: [['3', '1'], ['2', '2'], ['1', '0'], ['0', '3']],
 6: [['3', '0'], ['2', '2'], ['1', '3'], ['0', '1']],
 7: [['3', '0'], ['2', '3'], ['1', '2'], ['0', '1']],
 8: [['3', '0'], ['2', '3'], ['1', '1'], ['0', '2']],
 9: [['3', '0'], ['2', '1'], ['1', '3'], ['0', '2']],
 10: [['3', '0'], ['2', '1'], ['1', '2'], ['0', '3']],
 11: [['3', '0'], ['2', '2'], ['1', '1'], ['0', '3']]}

In [83]:
refTopologies={}
refTopologies['A']=[['1','0'],['0','2'],['2','3'],['3','1']]
refTopologies['A-bwd']=[['0','1'],['1','3'],['3','2'],['2','0']]
refTopologies['B']=[['1','0'],['0','3'],['3','2'],['2','1']]
refTopologies['B-bwd']=[['0','1'],['1','2'],['2','3'],['3','0']]
refTopologies['C']=[['2','0'],['0','3'],['3','1'],['1','2']]
refTopologies['C-bwd']=[['0','2'],['2','1'],['1','3'],['3','0']]
refTopologies['D']=[['0','3'],['3','0'],['2','1'],['1','2']]
refTopologies['El']=[['2','2'],['3','1'],['1','0'],['0','3']]
refTopologies['El-bwd']=[['2','2'],['1','3'],['3','0'],['0','1']]
refTopologies['Er']=[['1','1'],['3','2'],['2','0'],['0','3']]
refTopologies['Er-bwd']=[['1','1'],['2','3'],['3','0'],['0','2']]
refTopologies['F']=[['0','3'],['3','0'],['1','1'],['2','2']]

In [84]:
def swap(list, i, j):
    list[i], list[j] = list[j], list[i]
    return list

for idx,d in diagrams.items():
    for p in range(len(d)):
        if d[p][1] != d[(p+1)%len(d)][0]:
            for pp in range(p,len(d)):
                if d[p][1] == d[pp][0]:
                    swap(d,(p+1)%len(d),pp)
                    

In [85]:
from src.wick.utilities import permutations

def cyclic_permutations(list):
    return [[list[i-j] for i in range(len(list))] for j in range(len(list))]

print('\\begin{tabular}{c | c}')
print('d & Diagrams\\\\')
print('\\hline')
for idx,d in diagrams.items():
    for label, top in refTopologies.items():
        if d in permutations(top):
            print('{} & {}\\\\'.format(idx,label[0]))
print('\\end{tabular}')

\begin{tabular}{c | c}
d & Diagrams\\
\hline
0 & C\\
1 & B\\
2 & A\\
3 & E\\
4 & A\\
5 & E\\
6 & E\\
7 & B\\
8 & E\\
9 & C\\
10 & D\\
11 & F\\
\end{tabular}
